In [1]:
# Summary: What This Code Does
# Connects to MySQL.
# Runs a filtered and grouped query on the ws_hub.ipg_ez table.
# Retrieves only the latest report run date (max(rpt_run_date)) per group.
# Filters only records where Truck_Appointment_Date is present and rpt_run_time = '16:00:00'.
# Converts the result into a Pandas DataFrame.
# Displays the first five rows for validation.

from mysql_connector import fetch_data

qry = f"""

WITH MaxDate AS (
    SELECT 
        Site,
        BL_Number,
        State AS Ship_to_State,
        Ship_to_City,
        Ship_to_Customer,
        MAX(Truck_Appointment_Date) AS Truck_Appointment_Date,  -- Ensuring latest appointment date
        MAX(rpt_run_date) AS rpt_run_date,  -- Ensuring latest report run date
        rpt_run_time,
        Product_Group
    FROM ws_hub.ipg_ez
    WHERE Truck_Appointment_Date IS NOT NULL 
          AND rpt_run_time = '16:00:00'
          
    GROUP BY 
        Site, BL_Number, State, Ship_to_City, Ship_to_Customer, 
        rpt_run_time, Product_Group  -- Fixed GROUP BY
),
RankedData AS (
    SELECT 
        md.Site,
        md.BL_Number,
        md.Truck_Appointment_Date,
        ipg.BL_weight,
        ipg.Freight_Amount,
        md.Ship_to_State,
        md.Ship_to_City,
        md.Ship_to_Customer,
        md.rpt_run_date,
        md.rpt_run_time,
        md.Product_Group,
        ROW_NUMBER() OVER (
            PARTITION BY md.Site, md.BL_Number, md.Truck_Appointment_Date, md.Product_Group
            ORDER BY md.rpt_run_date DESC
        ) AS rn
    FROM MaxDate md
    JOIN ws_hub.ipg_ez ipg
    ON md.Site = ipg.Site 
    AND md.BL_Number = ipg.BL_Number 
    AND md.Truck_Appointment_Date = ipg.Truck_Appointment_Date
    AND md.rpt_run_date = ipg.rpt_run_date
    WHERE ipg.rpt_run_time = '16:00:00'
)
SELECT 
    Site,
    BL_Number,
    Truck_Appointment_Date,
    BL_weight,
    Freight_Amount,
    Ship_to_State,
    Ship_to_City,
    Ship_to_Customer,
    rpt_run_date,
    rpt_run_time,
    Product_Group
FROM RankedData
WHERE rn = 1;


"""

df = fetch_data(qry, database_name="ws_hub")
# print(df.head())

# Extract unique values based on the required columns
unique_df = df[['Site', 'BL_Number', 'Truck_Appointment_Date', 'BL_weight', 'Freight_Amount', 'Product_Group']].drop_duplicates()

# Display the unique values
# print(unique_df)

unique_df.to_csv('data/shipment_unique.csv', index=False)

# Group by Site, Product_Group, and Truck_Appointment_Date, then sum BL_weight and Freight_Amount
grouped_df = df.groupby(['Site', 'Product_Group', 'Truck_Appointment_Date'])[['BL_weight', 'Freight_Amount']].sum().reset_index()

# Display the summarized DataFrame
# print(grouped_df)

grouped_df.to_csv('data/shipment_grouped.csv', index=False)

[2025-02-07 13:41:56] Connected to database 'ws_hub' using section 'ws_hub'


In [ ]:
import pandas as pd
import plotly.express as px


# Load the dataset
# df = pd.read_csv("shipment_grouped.csv")  # Adjust path if needed

df = grouped_df.copy()

# Convert Truck_Appointment_Date to datetime
df["Truck_Appointment_Date"] = pd.to_datetime(df["Truck_Appointment_Date"])

# Filter for specific Site and Product_Group
filtered_df = df[(df["Site"] == "AMJK") & (df["Product_Group"] == "SW")].copy()

# Extract Year and Month
filtered_df["Year"] = filtered_df["Truck_Appointment_Date"].dt.year
filtered_df["Month"] = filtered_df["Truck_Appointment_Date"].dt.month

# Aggregate the data by Year, Month
aggregated_df = filtered_df.groupby(["Year", "Month"])[["BL_weight"]].sum().reset_index()

# Reshape data for Plotly
# melted_df = aggregated_df.melt(id_vars=["Year", "Month"], value_vars=["BL_weight"])

# Plot using Plotly
fig = px.line(
    aggregated_df,
    x="Month",
    y="BL_weight",
    color="Year",
    markers=True,
    title="IPG EZ Total Weight by Year Month",
    labels={"Month": "Month", "Weight": "Weight", "Year": "Year"},
)

# Show plot
fig.show()
